In [ ]:
# default_exp VideoCopy

# VideoCopy
## Will copy a video provided either a db entry or video link and a target server

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

### Required libraries

In [ ]:
#export
from pytube import YouTube
import psycopg2 as psql
import argparse

In [ ]:
#export
class Video:
    link=""
    workingdir = "."
    def start(self):
        print("ToDo")
        self.fixLink()
        
    def fixLink(self):
        pass
    


In [ ]:
#export
class YTVideo(Video):
    VID_ORDER = {0:'avc', 1:'vp9'}
    def __init__(self,filename = "temp", ytlink="", tube: YouTube = None):
        self.vid = None
        self.tempfilename = filename
        if not tube is None:
            self.vid = tube
        elif ytlink:
            self.link=ytlink
            self.fixLink()
            self.vid = YouTube(self.link)
            
        self.streams = {}
        self.captions = {}
        self.tempfiles = []
        
    def start(self):
        super().start()
    
    def fixLink(self):
        if not "https://www.youtube.com/" in self.link:
            print("Not YT")
    def parseStreams(self):
        self.streams['v'] = self.pickVid()
        self.streams['a'] = self.pickAudio()
        self.captions = self.parseCaptions()
        
        
    def pickVid(self):
        s = self.vid.streams

        #Filter out video streams, pick the "best" one by resolution, fps, and codec
        maxres = str(max([int((x.resolution or '000')[:-1]) for x in s.filter(only_video=True)]))+'p'
        #print("Max Res: ", maxres)
        maxfps = max([x.fps for x in s.filter(only_video=True,res=maxres)])
        #print("Max FPS: ", maxfps)
        cstreams = s.filter(only_video=True,res=maxres,fps=maxfps)
        #itags_bycodec = {x:0 for x in self.VID_ORDER.values()}
        #print(self.VID_ORDER)
        itag = 0
        for cd in self.VID_ORDER:
            if itag:
                break
            for st in cstreams:
                #print(self.VID_ORDER[cd])
                if self.VID_ORDER[cd] in st.video_codec:
                    #print("Picking: ", st.itag,st.video_codec)
                    itag = st.itag
                    break
                print (st.itag,st.video_codec)
        return s.itag_index[itag]

    def pickAudio(self):
        #Filter out audio stream, pick the "best" on by audio bit rate
        #picks first best codec, FIXME
        s = self.vid.streams
        maxadp = max([int(x.abr[:-4]) for x in s.filter(only_audio=True)])
        #print(maxadp)
        astreams = s.filter(only_audio=True, abr=str(maxadp)+"kbps")
        #print(astreams[0])
        return astreams[0]
    
    def parseCaptions(self):
        caps = {}
        for t in self.vid.caption_tracks:
            lang = t.code
            if "auto" in t.name:
                #print("auto")
                lang = 'a-' + lang
            #print(lang)
            caps[lang] = t
        return caps
    
    def downloadStreams(self):
        for s in self.streams:
            print (s,self.streams[s])
            self.streams[s].download()
        for c in self.captions:
            print(c,self.captions[c])
            self.captions[c].download(prefix=c,title = tempfilename,output_path = temppath)
        

### Use as a standalone/ on the command line
Or also from inside the notebook

In [ ]:
testlink="https://www.youtube.com/watch?v=S32L8ULLy94"
job = YTVideo(ytlink=testlink)
#job.parseStreams()
jobvid = job.vid
job.start()

ToDo


Polling youtube in the constructor takes the most time, reinitialize with the Youtube object and remake the class. Use this to update a class object and run commands against it.

In [ ]:
#hide
job = YTVideo(tube=job.vid)
#job.parseStreams()
job.parseStreams()
#job.downloadStreams()
print(job.workingdir)


.


In [ ]:
s = job.vid.streams
print([x for x in s.filter(only_video=True)])
maxres = max([int((x.resolution or '000')[:-1]) for x in s.filter(only_video=True)])

[<Stream: itag="137" mime_type="video/mp4" res="1080p" fps="30fps" vcodec="avc1.640028" progressive="False" type="video">, <Stream: itag="248" mime_type="video/webm" res="1080p" fps="30fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="399" mime_type="video/mp4" res="None" fps="30fps" vcodec="av01.0.08M.08" progressive="False" type="video">, <Stream: itag="136" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">, <Stream: itag="247" mime_type="video/webm" res="720p" fps="30fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="398" mime_type="video/mp4" res="None" fps="30fps" vcodec="av01.0.05M.08" progressive="False" type="video">, <Stream: itag="135" mime_type="video/mp4" res="480p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">, <Stream: itag="244" mime_type="video/webm" res="480p" fps="30fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="397" mime_type="video/mp4" res=

In [ ]:
for c, s in enumerate(job.vid.streams.filter(only_video=True)):
    print (c,s)

0 <Stream: itag="137" mime_type="video/mp4" res="1080p" fps="30fps" vcodec="avc1.640028" progressive="False" type="video">
1 <Stream: itag="248" mime_type="video/webm" res="1080p" fps="30fps" vcodec="vp9" progressive="False" type="video">
2 <Stream: itag="399" mime_type="video/mp4" res="None" fps="30fps" vcodec="av01.0.08M.08" progressive="False" type="video">
3 <Stream: itag="136" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">
4 <Stream: itag="247" mime_type="video/webm" res="720p" fps="30fps" vcodec="vp9" progressive="False" type="video">
5 <Stream: itag="398" mime_type="video/mp4" res="None" fps="30fps" vcodec="av01.0.05M.08" progressive="False" type="video">
6 <Stream: itag="135" mime_type="video/mp4" res="480p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">
7 <Stream: itag="244" mime_type="video/webm" res="480p" fps="30fps" vcodec="vp9" progressive="False" type="video">
8 <Stream: itag="397" mime_type="video/

In [ ]:
# export
def parseCLI():
    progname = "VideoCopy"
    par = argparse.ArgumentParser(progname)
    
    par.add_argument("-v", help="Link to video")
    args = par.parse_args()
    
    return None


In [ ]:
# export
# notest
#print(__name__)
if __name__ == "__main__":
    job = parseCLI()
    if job:
        pass
        #job.start    

usage: VideoCopy [-h] [-v V]
VideoCopy: error: unrecognized arguments: -f /home/jupyter/.local/share/jupyter/runtime/kernel-425e29af-6ea2-44ad-9980-00cc2db03914.json


SystemExit: 2